In [1]:
# Disable cuda
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [4]:
import utils
import torch

In [3]:
model = utils.give_model()

In [5]:
model.load_state_dict(torch.load("./model/model.pt", map_location="cpu"))

<All keys matched successfully>

In [7]:
train_dataloader, valid_dataloader, test_dataloader = utils.give_data_loaders()

In [8]:
batch = next(iter(test_dataloader))

In [10]:
# export model to onnx
torch.onnx.export(model, batch['image'][0].unsqueeze(0), "./model/model.onnx",
                    input_names=["input"], output_names=["output"], opset_version=11)


/home/venom/intel/oneapi/intelpython/latest/envs/pytorch/lib/python3.9/site-packages/segmentation_models_pytorch/base/model.py:16: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if torch.any((h % output_stride != 0) | (w % output_stride != 0)):


## OpenVino MO

In [9]:
from openvino.inference_engine import IECore

In [11]:
ie = IECore()
net = ie.read_network(model="./model/model.xml", weights="./model/model.bin")

exec_net = ie.load_network(network=net, device_name="CPU")


In [25]:
image = batch['image'][0].unsqueeze(0).numpy()
mask = batch['mask'][0].unsqueeze(0).numpy()

In [ ]:
criterion = utils.give_criterion()


In [26]:
# run inference
res = exec_net.infer(inputs={"input": image})

# check results
print("Dice score:", criterion(torch.tensor(
    res["output"]), torch.tensor(mask)).item())


In [38]:
# check time to inference, run 10 times
import timeit
timeit.timeit(lambda: exec_net.infer(inputs={"input": image}), number=1)

0.06846189200018671